### Imports necesarios

In [1]:
# import keras.losses
import numpy as np
import config
import pickle
from matplotlib import pyplot as plt
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.client import device_lib
# from tensorflow.keras.models import Sequential
# from keras import models
# from keras.utils import multi_gpu_model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
# tf.config.run_functions_eagerly(True)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import keras

### Cargar datos de entrenamiento

In [2]:
with open(config.TRAINING_FILE, "rb") as f:
    df = pickle.load(f)

df.head()

,image_name,Male
20803,20803.jpg,male
20820,20820.jpg,male
24617,24617.jpg,male
29604,29604.jpg,male
11602,11602.jpg,male


In [3]:
im = plt.imread(f"{config.TRAINING_IMGS}20803.jpg")
img_width, img_height = im.shape
pixel_count = img_width * img_height

img_names = df['image_name'].values.reshape(1, df['image_name'].values.size)[0]
X_train = np.array([plt.imread(f"{config.TRAINING_IMGS}{img_name}") for img_name in img_names]) / 255
Y_train = np.where(df["Male"] == "male", 1, 0).T

In [4]:
model = Sequential()
model.add(Conv2D(32, 3, padding="valid", activation="relu",  input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, 3, activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation="relu"))
model.add(Dense(2, activation="sigmoid"))

loss = keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = keras.optimizers.SGD(learning_rate=0.01)
metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 123008)            0         
                                                                 
 dense (Dense)               (None, 32)                3

In [ ]:
# tf.data.experimental.enable_debug_mode()
model.fit(X_train, one_hot(Y_train).T, verbose=1)

In [2]:
model.loss

NameError: name 'model' is not defined

In [5]:
def init_params():
    W1 = np.random.randn(2, pixel_count)
    b1 = np.random.randn(2, 1)
    W2 = np.random.randn(2, 2)
    b2 = np.random.randn(2, 1)
    return W1, b1, W2, b2

def ReLu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def forward_prop(X, W1, b1, W2, b2):
    Z1 = W1.dot(X) + b1
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    return np.vstack((np.array(np.where(Y == 1, 1, 0)), np.array(np.where(Y == 0, 1, 0))))

def deriv_ReLu(Z):
    return Z > 0

def back_prop(Z1, A1, A2, W2, X, Y):
    Y_one_hot = one_hot(Y)
    dZ2 = A2 - Y_one_hot
    dW2 = (1 / pixel_count) * dZ2.dot(A1.T)
    db2 = (1 / pixel_count) * np.sum(dZ2, axis=1).reshape(2, 1)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLu(Z1)
    dW1 = (1 / pixel_count) * dZ1.dot(X.T)
    db1 = (1 / pixel_count) * np.sum(dZ1, axis=1).reshape(2, 1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(X, W1, b1, W2, b2)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2,  dW1, db1, dW2, db2, alpha)
        if i % 100 == 0:
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_predictions(A2), Y)}")

    print("Done")
    return W1, b1, W2, b2

In [171]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 101, .1)

Iteration: 0
Accuracy: 0.5626386927577164


C:\Users\Santiago Quincke\AppData\Local\Temp\ipykernel_18560\1368660357.py:12: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
C:\Users\Santiago Quincke\AppData\Local\Temp\ipykernel_18560\1368660357.py:12: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(Z) / np.sum(np.exp(Z))


Iteration: 100
Accuracy: 0.5032277587250353
Done
